In [4]:
import glob
import pandas as pd
vote_resp = []

for file in sorted(glob.glob("published/medqa_gpt-4o-mini/*.csv")):
    print(file)
    df = pd.read_csv(file)
    vote_resp.append(df)    

published/medqa_gpt-4o-mini/0.csv
published/medqa_gpt-4o-mini/1.csv
published/medqa_gpt-4o-mini/10.csv
published/medqa_gpt-4o-mini/100.csv
published/medqa_gpt-4o-mini/1000.csv
published/medqa_gpt-4o-mini/1001.csv
published/medqa_gpt-4o-mini/1002.csv
published/medqa_gpt-4o-mini/1003.csv
published/medqa_gpt-4o-mini/1004.csv
published/medqa_gpt-4o-mini/1005.csv
published/medqa_gpt-4o-mini/1006.csv
published/medqa_gpt-4o-mini/1007.csv
published/medqa_gpt-4o-mini/1008.csv
published/medqa_gpt-4o-mini/1009.csv
published/medqa_gpt-4o-mini/101.csv
published/medqa_gpt-4o-mini/1010.csv
published/medqa_gpt-4o-mini/1011.csv
published/medqa_gpt-4o-mini/1012.csv
published/medqa_gpt-4o-mini/1013.csv
published/medqa_gpt-4o-mini/1014.csv
published/medqa_gpt-4o-mini/1015.csv
published/medqa_gpt-4o-mini/1016.csv
published/medqa_gpt-4o-mini/1017.csv
published/medqa_gpt-4o-mini/1018.csv
published/medqa_gpt-4o-mini/1019.csv
published/medqa_gpt-4o-mini/102.csv
published/medqa_gpt-4o-mini/1020.csv
published/me

In [6]:
vote_resp[0]

,Unnamed: 0,answer,token_usage
0,0,2.0,100
1,1,2.0,74
2,2,2.0,94
3,3,2.0,149
4,4,2.0,69
...,...,...,...
59,59,2.0,111
60,60,2.0,117
61,61,2.0,114
62,62,2.0,92


In [ ]:

df = pd.read_json('./resources/data/medqa.jsonl', lines=True)
letter_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
real_ans = df["answer_idx"].apply(lambda x: letter_to_number.get(x))

In [9]:
for df in vote_resp:
    vote_ans = int(df.answer.mode()[0])
    print(vote_ans)

2
4
5
3
3
3
3
3
4
5
3
2
2
4
1
3
5
1
4
3
1
4
5
2
1
1
4
4
1
3
3
4
1
2
5
1
1
3
5
3
2
3
2
2
4
1
2
3
2
4
5
5
2
1
5
2
2
3
2
4
4
2
2
2
3
4
1
1
1
2
2
5
2
4
5
1
1
3
2
2
4
5
1
4
5
3
5
5
4
2
5
4
5
5
3
1
2
1
2
2
2
3
5
4
4
3
4
3
4
4
3
5
1
4
4
1
1
4
4
2
1
2
1
4
2
3
4
3
3
5
2
5
4
1
4
4
1
4
1
1
5
5
4
1
3
5
3
4
1
1
3
3
1
3
5
4
5
1
3
1
3
2
5
3
1
3
1
1
2
5
1
5
2
1
4
5
3
5
1
1
4
3
1
2
5
3
2
4
3
1
3
4
1
4
5
1
1
2
4
3
2
3
5
1
1
1
4
5
3
3
2
3
4
3
5
3
1
4
5
2
4
3
3
1
2
4
2
4
1
3
1
5
5
2
4
1
2
3
1
2
5
1
1
5
5
1
3
5
2
3
3
1
3
2
5
5
3
5
1
5
1
2
2
3
1
3
1
4
2
2
1
1
3
5
1
4
3
1
5
4
1
5
1
2
2
2
1
2
4
1
3
1
2
4
1
1
2
3
3
2
5
1
4
2
3
3
3
1
5
2
2
1
2
1
3
4
3
3
2
4
4
1
1
2
4
5
4
4
2
1
2
2
5
1
1
4
5
2
1
4
2
5
5
2
4
1
1
3
2
4
2
4
5
3
2
3
1
2
4
5
2
4
4
2
4
5
2
3
2
1
3
2
5
3
3
1
2
5
3
3
3
5
3
3
4
3
3
2
2
3
2
1
2
4
1
2
1
1
3
4
1
1
2
1
3
4
5
1
4
4
1
5
1
3
3
4
2
2
2
1
3
3
4
2
5
2
1
5
2
1
1
5
1
1
5
3
1
2
4
4
2
5
1
2
1
4
1
3
4
1
1
4
5
3
3
4
3
1
2
2
5
5
4
1
3
4
3
1
1
1
3
4
1
2
4
2
4
1
1
4
5
4
1
2
4
4
2
1
2
1
1
2
1
2
5
2
2
5
2
2


In [13]:
sum(real_ans == vote_ans) / len(real_ans)

0.211311861743912

In [14]:
import typing
import sys

import pydantic
import langchain_openai
import langchain_core
import tqdm.auto
import pandas as pd

from consol.confidence_models import AbstractConfidenceModel, SbftConfidenceModel, SprtConfidenceModel, PValueConfidenceModel, BayesianConfidenceModel, VoteConfidenceModel

class ConfidentSolverConfig(pydantic.BaseModel):
    max_trials: int

class ConfidentSolver:
    def __init__(
        self,
        confidence_model: typing.Union[str, AbstractConfidenceModel],
        max_trials=64,
    ):
        self.config = ConfidentSolverConfig(
            max_trials=max_trials,
        )
        if confidence_model == "sbft":
            self.confidence_model = SbftConfidenceModel()
        elif confidence_model == "sprt":
            self.confidence_model = SprtConfidenceModel()
        elif confidence_model == "pvalue":
            self.confidence_model = PValueConfidenceModel()
        elif confidence_model == "bayesian":
            self.confidence_model = BayesianConfidenceModel()
        elif confidence_model == "vote":
            self.confidence_model = VoteConfidenceModel()
        elif isinstance(confidence_model, AbstractConfidenceModel):
            self.confidence_model = confidence_model
        else:
            raise ValueError(f"Unknown Confidence Model: {confidence_model}")

    def invoke(self, answers, debug=False):
        max_trials = self.config.max_trials

        total_raw_outputs = []
        total_raw_outputs.append(answers[0]) 
        i = 1

        total_invoke = 0
        while True:     
            total_invoke += 1       
            first, second = self._get_top_two_answers(total_raw_outputs)
            trials = self._determine_trials(first, second, max_trials, len(total_raw_outputs))
            if trials == 0:
                break
            for j in range(trials):
                total_raw_outputs.append(answers[i])
                i += 1
        df = self._create_dataframe(total_raw_outputs)
        if debug:
            return df
        return {"target": df['answer'].mode().iloc[0], "total_invoke": total_invoke, "total_runs": len(total_raw_outputs)}

    def _get_top_two_answers(self, total_raw_outputs):
        total_ss = pd.Series(total_raw_outputs).value_counts()
        two = total_ss.sort_values(ascending=False).head(2).to_list()
        while len(two) < 2:
            two += [0]
        return two[0], two[1]

    def _determine_trials(self, first, second, max_trials, current_trials):
        for trials in range(0, max_trials + 1):
            if first + trials == 0:
                continue
            if self.confidence_model.test(first + trials, second):
                break
        if trials >= max_trials - current_trials:
            trials = max_trials - current_trials
        return trials

    def _create_dataframe(self, total_raw_outputs):
        return pd.DataFrame({
            'answer': total_raw_outputs,
        })

/Users/2katz/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:

MAX_EXPR = 10_000
MAX_ITER = 64

records = []
for i, problem in tqdm.tqdm(enumerate(vote_resp)):
    problem = problem["answer"]
    answer = real_ans[i]

    for solver_name in ["sbft", "sprt", "pvalue","bayesian"]:
        solver = ConfidentSolver(solver_name)
        ret = solver.invoke(problem)
        ret["solver"] = solver_name
        ret["answer"] = answer
        ret["correct"] = ret["answer"] == ret["target"]
        records.append(ret)

1273it [00:03, 401.44it/s]


In [18]:
df = pd.DataFrame(records)
print(df)

      target  total_invoke  total_runs    solver  answer  correct
0        2.0             2           9      sbft       3    False
1        2.0             2           9      sprt       3    False
2        2.0             2           5    pvalue       3    False
3        2.0             2           4  bayesian       3    False
4        4.0             6          64      sbft       5    False
...      ...           ...         ...       ...     ...      ...
5087     2.0             2           4  bayesian       3    False
5088     4.0             4          21      sbft       3    False
5089     4.0             4          17      sprt       3    False
5090     4.0             3           8    pvalue       3    False
5091     4.0             2           4  bayesian       3    False

[5092 rows x 6 columns]


In [25]:
df.groupby("solver")["correct"].mean()

solver
bayesian    0.196386
pvalue      0.195601
sbft        0.195601
sprt        0.197958
Name: correct, dtype: float64

In [26]:

MAX_EXPR = 10_000
MAX_ITER = 64

records = []
for i, problem in tqdm.tqdm(enumerate(vote_resp)):
    problem = problem["answer"]
    answer = problem.mode()[0]

    for solver_name in ["sbft", "sprt", "pvalue","bayesian"]:
        solver = ConfidentSolver(solver_name)
        ret = solver.invoke(problem)
        ret["solver"] = solver_name
        ret["answer"] = answer
        ret["correct"] = ret["answer"] == ret["target"]
        records.append(ret)

1273it [00:03, 387.07it/s]


In [27]:
df = pd.DataFrame(records)
print(df)

      target  total_invoke  total_runs    solver  answer  correct
0        2.0             2           9      sbft     2.0     True
1        2.0             2           9      sprt     2.0     True
2        2.0             2           5    pvalue     2.0     True
3        2.0             2           4  bayesian     2.0     True
4        4.0             6          64      sbft     4.0     True
...      ...           ...         ...       ...     ...      ...
5087     2.0             2           4  bayesian     2.0     True
5088     4.0             4          21      sbft     4.0     True
5089     4.0             4          17      sprt     4.0     True
5090     4.0             3           8    pvalue     4.0     True
5091     4.0             2           4  bayesian     4.0     True

[5092 rows x 6 columns]


In [28]:
df.groupby("solver")["correct"].mean()

solver
bayesian    0.997643
pvalue      0.999214
sbft        1.000000
sprt        0.989788
Name: correct, dtype: float64

In [31]:
df.groupby("solver")["total_runs"].mean()

solver
bayesian     9.322859
pvalue      11.147683
sbft        19.833464
sprt        16.440691
Name: total_runs, dtype: float64